In [1]:
import pandas as pd

df = pd.read_csv("Rainfall.csv") 
df = df.dropna()

from IPython.display import display
display(df.head(20))

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
0,1,1025.9,19.9,18.3,16.8,13.1,72,49,yes,9.3,80.0,26.3
1,2,1022.0,21.7,18.9,17.2,15.6,81,83,yes,0.6,50.0,15.3
2,3,1019.7,20.3,19.3,18.0,18.4,95,91,yes,0.0,40.0,14.2
3,4,1018.9,22.3,20.6,19.1,18.8,90,88,yes,1.0,50.0,16.9
4,5,1015.9,21.3,20.7,20.2,19.9,95,81,yes,0.0,40.0,13.7
5,6,1018.8,24.3,20.9,19.2,18.0,84,51,yes,7.7,20.0,14.5
6,7,1021.8,21.4,18.8,17.0,15.0,79,56,no,3.4,30.0,21.5
7,8,1020.8,21.0,18.4,16.5,14.4,78,28,no,7.7,60.0,14.3
8,9,1020.6,18.9,18.1,17.1,14.3,78,79,no,3.3,70.0,39.3
9,10,1017.5,18.5,18.0,17.2,15.5,85,91,yes,0.0,70.0,37.7


In [2]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["rainfall"])
y = df["rainfall"]
y = y.map({"yes": 1, "no": 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [90]:
# from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier(n_estimators=1000, random_state=44)

# from sklearn.ensemble import GradientBoostingClassifier
# rf_model = GradientBoostingClassifier(subsample=0.5, n_estimators = 1000, learning_rate=0.01, random_state=44)

import xgboost as xgb
from xgboost import XGBClassifier
rf_model = XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=5, subsample=0.4, colsample_bytree=0.8, random_state=44)

rf_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [91]:
from sklearn.metrics import roc_auc_score, roc_curve

train_auc = roc_auc_score(y_train, rf_model.predict_proba(X_train)[:, 1])
test_auc = roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]) 

print(f"Train AUC: {train_auc:.4f}")
print(f"Test AUC: {test_auc:.4f}")

Train AUC: 0.9835
Test AUC: 0.7404


In [15]:
y_probs = rf_model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {auc:.4f}")

ROC AUC Score: 0.7545


In [5]:
print(y_probs)

[9.99977342e-01 9.93281497e-01 9.98932779e-01 9.70996767e-01
 3.56414002e-01 9.62513028e-01 9.99745264e-01 7.91195271e-01
 9.99962001e-01 9.94205310e-01 9.99916966e-01 4.68280720e-04
 1.64776140e-01 9.99946739e-01 4.95908537e-01 2.88434556e-02
 8.07230031e-05 9.99920596e-01 5.55472760e-04 9.99843352e-01
 9.98351780e-01 9.63584561e-01 9.98644392e-01 1.03001611e-01
 7.98582931e-03 9.99723158e-01 9.89813600e-01 2.17991969e-01
 3.52016304e-04 8.24622607e-01 9.52933170e-01 9.48045115e-01
 9.95178331e-01 9.95037382e-01 2.14360262e-02 9.96291984e-01
 9.96950787e-01 9.99998893e-01 9.99965383e-01 1.84687588e-02
 9.99639896e-01 9.99088748e-01 9.99703788e-01 9.99965003e-01
 9.99938120e-01 4.26920330e-04 8.52178819e-02 9.99988230e-01
 6.45517736e-03 9.99934348e-01 7.61230634e-03 9.95662845e-01
 9.99503158e-01 8.33801771e-01 6.65310036e-01 8.67012940e-01
 8.99679831e-01 9.92292674e-01 9.51547052e-01 9.99617148e-01
 9.99807353e-01 9.96027527e-01 9.97678272e-01 9.92189239e-01
 4.45629179e-03 9.997887